In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/food41")


Resuming download from 155189248 bytes (5538618858 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/kmader/food41?dataset_version_number=5 (155189248/5693808106) bytes left.


100%|██████████| 5.30G/5.30G [07:22<00:00, 12.5MB/s]

Extracting files...


Path to dataset files: /Users/ruoqiyan/Desktop/690ML/ml_project/data


In [5]:
print("Path to dataset files:",path)

Path to dataset files: /Users/ruoqiyan/.cache/kagglehub/datasets/kmader/food41/versions/5


In [6]:
path0 = '/Users/ruoqiyan/Desktop/690ML/ml_project/data'

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


### Data augmentation and normalization for training

In [20]:
# Set dataset path
dataset_path = '/Users/ruoqiyan/Desktop/690ML/ml_project/data/images/'

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize images to [0, 1]
    #rotation_range=20,  # Random rotations up to 20 degrees
    width_shift_range=0.2,  # Random width shifts
    height_shift_range=0.2,  # Random height shifts
    zoom_range=0.2,  # Random zooming
    #horizontal_flip=True,  # Flip images horizontally
    validation_split=0.2  # Split 20% for validation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='categorical',
    subset='training'  # 80% for training
)

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),  # Resize images to 128x128
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # 20% for validation
)



Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


build CNN model

In [21]:
max_pooling_stride = 4
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((max_pooling_stride, max_pooling_stride)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((max_pooling_stride, max_pooling_stride)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(101, activation='softmax')  # 101 categories
])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 31, 31, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 7, 7, 64)         

In [22]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
history = model.fit(
    train_generator,
    epochs=5,  # Adjust the number of epochs as needed
    validation_data=validation_generator
)

Epoch 1/5
 478/2525 [====>.........................] - ETA: 3:37 - loss: 4.5886 - accuracy: 0.0141

KeyboardInterrupt: 

In [17]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

632/632 [==============================] - 94s 148ms/step - loss: 4.0485 - accuracy: 0.1000
Test Accuracy: 10.00%


### Predict with a random picture of food

In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_food(img_path, model):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make a prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Get class labels
    class_labels = list(train_generator.class_indices.keys())
    print(f'Predicted Food: {class_labels[predicted_class]}')

# Test the function with a new image
predict_food('/Users/ruoqiyan/Desktop/690ML/ml_project/data/random_pics/9454-greek-baklava-s-tuttle-71eff77a5e3e4ceb94c1d48bafec55f1.webp', model)


1/1 [==============================] - 0s 17ms/step
Predicted Food: hot_and_sour_soup


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper left')
plt.show()


In [ ]:
# Save the model
model.save('food_recognition_model.h5')

# Load the model
from tensorflow.keras.models import load_model
loaded_model = load_model('food_recognition_model.h5')
